<a href="https://colab.research.google.com/github/Binaz/AddressParser/blob/main/Address_Parser_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install evaluate transformers==4.49.0
!pip install datasets==2.14.6
!pip install DatasetDict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
import os

# Use os.path to handle file paths
csv_path = os.path.join("/content/drive", "My Drive", "Colab Notebooks", "Version 3","Training_Dataset_ForModelVersion_3_2.csv")

# Load the dataset using the correct path

address_ner_dataset = load_dataset('csv',data_files=csv_path )  #TrevorJS/filtered-deep-parse-us-addresses

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.12/dist-packages/datasets/download/streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


In [ ]:
# Fix: Convert stringified lists to actual lists if needed
def convert_str_lists(example):
    if isinstance(example["tokens"], str):
        example["tokens"] = eval(example["tokens"])
    if isinstance(example["ner_tags"], str):
        example["ner_tags"] = eval(example["ner_tags"])
    return example

address_ner_dataset = address_ner_dataset.map(convert_str_lists)

Map:   0%|          | 0/210572 [00:00<?, ? examples/s]

In [ ]:
## This function randomly selects / splits records for training , validation and test dataset.

# Split the dataset into train, validation, and test sets
# First split into train and temp (this will be used for validation + test)
train_dataset, temp_dataset = address_ner_dataset["train"].train_test_split(test_size=0.2).values()

# Then split temp into validation and test
validation_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.5).values()

# Now you have `train_dataset`, `validation_dataset`, and `test_dataset`

# If needed, you can print to confirm the splits
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(validation_dataset)}")
print(f"Test set size: {len(test_dataset)}")

In [ ]:
# The first 80% goes to training (by order),
# The last 20% is split evenly into validation and test (10% each), randomly.

from datasets import Dataset, DatasetDict

# Access the actual dataset from the DatasetDict
full_dataset = address_ner_dataset["train"]

# Define split percentage
train_split_percent = 0.8
total_samples = len(full_dataset)
train_end = int(total_samples * train_split_percent)

# Deterministic slicing
train_dataset = full_dataset.select(range(train_end))
remaining_dataset = full_dataset.select(range(train_end, total_samples))

# Now split the remaining into validation and test
val_test_split = remaining_dataset.train_test_split(test_size=0.5, seed=42)
validation_dataset = val_test_split['train']
test_dataset = val_test_split['test']

# Optional: Group into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})


In [ ]:
print(train_dataset.features)
print(train_dataset[0:50])
print(validation_dataset[0:50])
print(test_dataset[0:50])

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}
{'tokens': [['Carrier', 'Release', 'Authorized', 'Kirat', 'Zaeske', '62406', 'Coniferous', 'Drive', 'KIRBY', ',', 'VERMONT', '05851'], ['Maxxis', 'Feldstein', 'Tax', 'Documents', 'Enclosed', '34004', 'Marblehead', 'Court', 'PARADISE', 'UT', '84328', '-', '8001', 'Eco-Friendly', 'Packaging'], ['Mahaylia', 'Drinkwine', '37210', 'Millbank', 'Court', 'RAY', 'CITY', ',', 'GEORGIA', 'Serial', 'Number'], ['Navya', 'Herrlich', '725', 'Brookstone', 'Drive', 'Ne', 'Delivery', 'Instructions', 'Attached', 'BLOOMSBURY', ',', 'NEW', 'JERSEY', '08804', '-', '9998', 'Tamper', 'Evident'], ['Meldoh', 'Ingran', '7182', 'Chalfont', 'Avenue', 'LACONA', ',', 'OSWEGO', 'NY', '13083', '-', '9998'], ['Wedding', 'Invitation', 'Sofronio', 'Gregware', '49930', 'Cochranton', 'Road', 'Apartment', '70', 'ANNA', 'MARIA', 'FLORIDA'], ['Radyah', 'Desnoyer', '

In [ ]:
# Accessing the label names from the 'ner_tags' feature.   This is used
label_names = ['O','B-NAME','I-NAME','B-STREET_NUM','I-STREET_NUM','B-STREET','I-STREET','B-UNIT','I-UNIT','B-CITY','I-CITY','B-STATE','I-STATE','B-POSTAL','I-POSTAL'] #address_ner_dataset['train'][0]['ner_tags'] #tokenized_datasets['train'].features['ner_tags']

#label_encoding_dict = {'O':0,'B-NAME':1,'I-NAME':2,'B-STREET_NUMBER':3,'I-STREET_NUMBER':4,'B-STREET_NAME':5,'I-STREET_NAME':6,'B-UNIT_NUMBER':7,'I-UNIT_NUMBER':8,'B-UNIT_DESIGNATOR':9,'I-UNIT_DESIGNATOR':10,'B-CITY':11,'I-CITY':12,'B-STATE_ABBREVIATION':13,'I-STATE_ABBREVIATION':14,'B-PLUS_4':15,'I-PLUS_4':16,'B-STATE_NAME':17,'I-STATE_NAME':18,'B-POSTAL_CODE':19,'I-POSTAL_CODE':20,'B-TRACKING_NUMBER':21,'I-TRACKING_NUMBER':22}
label_encoding_dict = {
    "O": 0,
    "B-NAME": 1,
    "I-NAME": 2,
    "B-STREET_NUM": 3,
    "I-STREET_NUM": 4,
    "B-STREET": 5,
    "I-STREET": 6,
    "B-UNIT": 7,
    "I-UNIT": 8,
    "B-CITY": 9,
    "I-CITY": 10,
    "B-STATE": 11,
    "I-STATE": 12,
    "B-POSTAL": 13,
    "I-POSTAL": 14
}
label_names

['O',
 'B-NAME',
 'I-NAME',
 'B-STREET_NUM',
 'I-STREET_NUM',
 'B-STREET',
 'I-STREET',
 'B-UNIT',
 'I-UNIT',
 'B-CITY',
 'I-CITY',
 'B-STATE',
 'I-STATE',
 'B-POSTAL',
 'I-POSTAL']

In [ ]:
train_dataset[0]

{'tokens': ['Carrier',
  'Release',
  'Authorized',
  'Kirat',
  'Zaeske',
  '62406',
  'Coniferous',
  'Drive',
  'KIRBY',
  ',',
  'VERMONT',
  '05851'],
 'ner_tags': ['O',
  'O',
  'O',
  'B-NAME',
  'I-NAME',
  'B-STREET_NUM',
  'B-STREET',
  'I-STREET',
  'B-CITY',
  'O',
  'B-STATE',
  'B-POSTAL']}

In [ ]:
def tokenize_and_align_labels(batch):
    label_all_tokens = True
    #tokenized_inputs = tokenizer(list(batch["tokens"]), truncation=True, is_split_into_words=True, padding='max_length')
    tokenized_inputs = tokenizer(batch['tokens'], truncation=True, is_split_into_words=True, padding='max_length') #return_offsets_mapping=True

     # Define a mapping from beginning (B-) labels to inside (I-) labels
    begin2inside = {
        "B-NAME": "I-NAME",
        "B-STREET_NUM": "I-STREET_NUM",
        "B-STREET": "I-STREET",
        "B-UNIT": "I-UNIT",
        "B-CITY" : "I-CITY",
        "B-STATE" : "I-STATE",
        "B-POSTAL" : "I-POSTAL",
    }

    labels = []
    for i, label in enumerate(batch[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                labelId = (label_encoding_dict[label[word_idx]])
                # Change B- to I- if the previous word is the same
                if label[word_idx] in begin2inside:
                    labelId = (label_encoding_dict[begin2inside[label[word_idx]]])  # Map B- to I-
                label_ids.append(labelId if label_all_tokens else -100)

            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
from transformers import AutoTokenizer

# Define the checkpoint you want to use for the tokenizer.
checkpoint = 'dslim/bert-base-NER-uncased'

tokenizer = AutoTokenizer.from_pretrained(checkpoint,num_labels=len(label_names),is_split_into_words = True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
## The follwoing code is to test if tokenize_and_align_labels is working correctly

# Example input (word-level tokens with labels)
#test_batch = {
#    "tokens": [["19", "Waldo", "AvenuewESTeAST","East", "BelfastCORNILANA", ",", "ME", "04915" ,"Consolidated", "Communications"]],
#    "ner_tags": [[
#        "S-STREET_NUM", "B-STREET", "I-STREET", "E-STREET", "S-CITY", "O", "S-STATE", "S-POSTAL","O","O"
#    ]]
#}

# Take a single example from your actual dataset
test_example = train_dataset[3]


# Wrap it in a batch format expected by the function
test_batch = {
    "tokens": [test_example["tokens"]],
    "ner_tags": [test_example["ner_tags"]]
}

#label_encoding_dict = {'O':0,'B-NAME':1,'I-NAME':2,'B-STREET_NUMBER':3,'I-STREET_NUMBER':4,'B-STREET_NAME':5,'I-STREET_NAME':6,'B-UNIT_NUMBER':7,'I-UNIT_NUMBER':8,'B-UNIT_DESIGNATOR':9,'I-UNIT_DESIGNATOR':10,'B-CITY':11,'I-CITY':12,'B-STATE_ABBREVIATION':13,'I-STATE_ABBREVIATION':14,'B-PLUS_4':15,'I-PLUS_4':16,'B-STATE_NAME':17,'I-STATE_NAME':18,'B-POSTAL_CODE':19,'I-POSTAL_CODE':20,'B-TRACKING_NUMBER':21,'I-TRACKING_NUMBER':22}
label_encoding_dict = {
    "O": 0,
    "B-NAME": 1,
    "I-NAME": 2,
    "B-STREET_NUM": 3,
    "I-STREET_NUM": 4,
    "B-STREET": 5,
    "I-STREET": 6,
    "B-UNIT": 7,
    "I-UNIT": 8,
    "B-CITY": 9,
    "I-CITY": 10,
    "B-STATE": 11,
    "I-STATE": 12,
    "B-POSTAL": 13,
    "I-POSTAL": 14
}
# Reverse mapping
id2label = {v: k for k, v in label_encoding_dict.items()}

# Use your tokenize_and_align_labels function
# Apply the function
encoded = tokenize_and_align_labels(test_batch)

# Visualize token-label mapping
tokens = tokenizer.convert_ids_to_tokens(encoded["input_ids"][0])
labels = encoded["labels"][0]

print("Token\t\tLabel")
print("-" * 30)

for token, label_id in zip(tokens, labels):
    if label_id == -100:
        print(f"{token:10s}\tIGNORED")
    else:
        print(f"{token:10s}\t{id2label[label_id]}")


Token		Label
------------------------------
[CLS]     	IGNORED
navy      	B-NAME
##a       	I-NAME
herr      	I-NAME
##lich    	I-NAME
72        	B-STREET_NUM
##5       	I-STREET_NUM
brooks    	B-STREET
##tone    	I-STREET
drive     	I-STREET
ne        	I-STREET
delivery  	O
instructions	O
attached  	O
blooms    	B-CITY
##bury    	I-CITY
,         	O
new       	B-STATE
jersey    	I-STATE
08        	B-POSTAL
##80      	I-POSTAL
##4       	I-POSTAL
-         	I-POSTAL
999       	I-POSTAL
##8       	I-POSTAL
tam       	O
##per     	O
evident   	O
[SEP]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IGNORED
[PAD]     	IG

In [ ]:
# Tokenize the first training example from the dataset
token = tokenizer(train_dataset[0]['tokens'],is_split_into_words = True)  ##, return_offsets_mapping=True

# Print the tokenizer object, the tokenized tokens, and the word IDs
print(token, '\n--------------------------------------------------------------------------------------\n',
      token.tokens(),'\n--------------------------------------------------------------------------------------\n',
      token.word_ids(),'\n--------------------------------------------------------------------------------------\n')

NameError: name 'train_dataset' is not defined

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=train_dataset.column_names)
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=test_dataset.column_names)
tokenized_validation_dataset = validation_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=validation_dataset.column_names)

Map:   0%|          | 0/168457 [00:00<?, ? examples/s]

Map:   0%|          | 0/21058 [00:00<?, ? examples/s]

Map:   0%|          | 0/21057 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

# Create a DataCollatorForTokenClassification object
data_collator = DataCollatorForTokenClassification(tokenizer)
print(tokenized_train_dataset)
# Testing data using the data collator
batch = data_collator([tokenized_train_dataset[i] for i in range(1)])

# Display the resulting batch
batch

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 168457
})


{'input_ids': tensor([[  101,  6839,  2713,  9362, 15163,  2102, 23564,  2229,  3489,  5786,
         12740,  2575,  9530, 23930,  3298, 15129,  1010,  8839,  5709, 27531,
          2487,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
# Install the seqeval library for evaluating sequence tasks
!pip install seqeval ;
!pip install evaluate ;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=abe6fdd10a3d1ecad708ab5f87a4a9b9b2885165bc5f39bd8c76b245726d32ce
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [ ]:
# Import the seqeval metric from Hugging Face's datasets library
import evaluate

# Load the seqeval metric which can evaluate NER and other sequence tasks
metric = evaluate.load("seqeval")

In [ ]:
import numpy as np
# Function to compute evaluation metrics from model logits and true labels
def compute_metrics(logits_and_labels):

  # Unpack the logits and labels
  logits, labels = logits_and_labels

  # Get predictions from the logits
  predictions = np.argmax(logits, axis=-1)

  # Remove ignored index (special tokens)
  str_labels = [
    [label_names[t] for t in label if t!=-100] for label in labels
  ]

  str_preds = [
    [label_names[p] for (p, t) in zip(prediction, label) if t != -100]
    for prediction, label in zip(predictions, labels)
  ]

  # Compute metrics
  results = metric.compute(predictions=str_preds, references=str_labels)

  # Extract key metrics
  return {
    "precision": results["overall_precision"],
    "recall": results["overall_recall"],
    "f1": results["overall_f1"],
    "accuracy": results["overall_accuracy"]
  }

In [ ]:
# Create mapping from label ID to label string name
id2label = {k: v for k, v in enumerate(label_names)}

# Create reverse mapping from label name to label ID
label2id = {v: k for k, v in enumerate(label_names)}

print(id2label , '\n--------------------\n' , label2id)

{0: 'O', 1: 'B-NAME', 2: 'I-NAME', 3: 'B-STREET_NUM', 4: 'I-STREET_NUM', 5: 'B-STREET', 6: 'I-STREET', 7: 'B-UNIT', 8: 'I-UNIT', 9: 'B-CITY', 10: 'I-CITY', 11: 'B-STATE', 12: 'I-STATE', 13: 'B-POSTAL', 14: 'I-POSTAL'} 
--------------------
 {'O': 0, 'B-NAME': 1, 'I-NAME': 2, 'B-STREET_NUM': 3, 'I-STREET_NUM': 4, 'B-STREET': 5, 'I-STREET': 6, 'B-UNIT': 7, 'I-UNIT': 8, 'B-CITY': 9, 'I-CITY': 10, 'B-STATE': 11, 'I-STATE': 12, 'B-POSTAL': 13, 'I-POSTAL': 14}


In [ ]:
# Load pretrained token classification model from Transformers
from transformers import AutoModelForTokenClassification

# Initialize model object with pretrained weights
model = AutoModelForTokenClassification.from_pretrained(
  checkpoint,
  num_labels=len(label_names),
  # Pass in label mappings
  id2label=id2label,
  label2id=label2id,
  ignore_mismatched_sizes=True
)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER-uncased and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torc

In [ ]:
from transformers import (
    Trainer, TrainingArguments, EarlyStoppingCallback,
    AutoModelForTokenClassification, get_scheduler, AdamW
)

# Create mapping from label ID to string and vice versa
id2label = {k: v for k, v in enumerate(label_names)}
label2id = {v: k for k, v in enumerate(label_names)}

# Load model with label mapping
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

# Training arguments
training_args = TrainingArguments(
    output_dir="address_parser_fine_tuned_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,               #Lower the better, can range from 1e-5 (i.e. 0.00001 small) to 2e-5(i.e. 0.00002 large)
    per_device_train_batch_size=32,   # For less number of training records (eg. <5k), lesser the batch size the better like 16. But for greater training dataset batch size should be more like 32
    per_device_eval_batch_size=32,
    num_train_epochs=15,
    weight_decay=0.01,
    gradient_accumulation_steps=1,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Must match the key returned by compute_metrics
    report_to="none",
    fp16=True                    # Enable mixed-precision training
)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Scheduler setup
num_training_steps = (
    (len(tokenized_train_dataset) // training_args.per_device_train_batch_size)
    * training_args.num_train_epochs
)

# Compute warmup steps as 10% of total steps
num_warmup_steps = int(0.1 * num_training_steps)

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

# Early stopping callback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)  # Early Stopping after 3 epochs

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset.shuffle(seed=42).select(range(168456)),
    eval_dataset=tokenized_test_dataset.shuffle(seed=42).select(range(21056)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    optimizers=(optimizer, lr_scheduler),  # Correct way to use scheduler
    callbacks=[early_stopping_callback],
)

# Start training
trainer.train()

path = F"/content/drive/My Drive/Colab Notebooks/Version 3/Trainer_Model_Version_3_2/"
trainer.save_model(path)


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER-uncased and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torc

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.011300,0.011767,0.991831,0.994034,0.992931,0.997278
2,0.008200,0.008650,0.994581,0.995542,0.995061,0.997995
3,0.003800,0.006779,0.995614,0.996483,0.996048,0.998488
4,0.002600,0.006192,0.996628,0.997076,0.996852,0.998688
5,0.002000,0.006190,0.996874,0.997339,0.997106,0.998782
6,0.001400,0.006331,0.996595,0.997280,0.996937,0.998809
7,0.000300,0.006316,0.997154,0.997559,0.997356,0.998834
8,0.000400,0.006773,0.997374,0.997678,0.997526,0.998956
9,0.000100,0.007125,0.997255,0.997610,0.997433,0.998970
10,0.000000,0.008026,0.997289,0.997763,0.997526,0.998912


In [ ]:
from transformers import pipeline

nerTrainer_new = pipeline(
    'token-classification',
    '/content/drive/My Drive/Colab Notebooks/Version 3/Trainer_Model_Version_3_2/',
    tokenizer=tokenizer,
    grouped_entities=True,
    aggregation_strategy = 'simple' ,
    device = -1   # device = 0 for gpu & device = -1 for cpu
)